# 第 2 章 AI 聊天功能的基礎–Respsones API

In [ ]:
from google.colab import userdata
from rich.pretty import pprint
import openai
client = openai.OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

## 2-1 更換模型--使用推理模型

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="3.8 和 3.11 哪一個大？"
)
print(response.output_text)

In [ ]:
response = client.responses.create(
    model="o3-mini",
    input="3.8 和 3.11 哪一個大？"
)
print(response.output_text)

In [ ]:
pprint(response)

### 調整推理強度

In [ ]:
logic_problem = """
C 說他的年齡是以下其中一個：

```
35 36 38
42 45 46
51 55 57
61 62
```

他將十位數告訴 A、個位數告訴 B。

A 看了這 11 個數後說：『我不知道 C 的年齡, 但我認為 B 也不知道。』

B 聽完再看了這 11 個數後說：『原本我並不知道 C 的年齡, 但現在我知道了。』

A 聽了 B 這樣說, 再看看這 11 個數, 說：『那我也知道 C 的年齡了。』

請問 C 的年齡是？
"""

In [ ]:
import time
start = time.time()
response = client.responses.create(
    model="o3-mini",
    input=logic_problem,
    reasoning={ # 不具推理能力的模型不支援
        "effort": 'high'
    }
)
end = time.time()

print(response.output_text)

In [ ]:
print(f"耗時 {end - start} 秒")
print(response.usage.output_tokens_details.reasoning_tokens)

### 觀察推理過程

#### 驗證組織

要顯示推理細節，需要先[驗證 API 使用者所屬的組織](https://platform.openai.com/settings/organization/general)

#### 顯示推理過程

In [ ]:
response = client.responses.create(
    model="o3-mini",
    input=logic_problem,
    reasoning={ # 不具推理能力的模型不支援
        "summary": 'auto' # auto、concise 或 detailed
    }
)

pprint(response)

In [ ]:
print(response.output_text)

## 2-2 控制生成結果

### 控制生成的 token 數量

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="你知道什麼是藍眼淚嗎？",
    # 可生成的 token 數量上限，包含推理的用量在內
    max_output_tokens=20 # 最少 16
)

print(response.output_text)

In [ ]:
pprint(response)

In [ ]:
response = client.responses.create(
    model="o3-mini",
    input="3.11 和 3.8 哪個大？",
    max_output_tokens=40
)
print(response.output_text)
pprint(response.usage)

### 控制回覆內容的變化性--temperature

In [ ]:
for _ in range(4):
    response = client.responses.create(
        model="gpt-4.1-nano",
        input="嗨！",
        # 0~2，溫度越高越活潑、天馬行空，
        # 0 表示不滾動，只會選到最高分的 token
        temperature=0,
    )

    print(response.output_text)

In [ ]:
for _ in range(4):
    response = client.responses.create(
        model="gpt-4.1-nano",
        input="嗨！",
        # 溫度設過高會選到奇怪的 token，
        # 導致後面接龍一路歪樓
        temperature=1.9,
        max_output_tokens=100
    )

    print(response.output_text)

### 控制生成內容的可能性--top_p

In [ ]:
for _ in range(4):
    response = client.responses.create(
        model="gpt-4.1-nano",
        input="嗨！",
        temperature=1.9,
        # 0~1，表示百分比，從分數高的往分數低
        # 的多少百分比放入候選池中
        top_p=0
    )

    print(response.output_text)

In [ ]:
for _ in range(4):
    response = client.responses.create(
        model="gpt-4.1-nano",
        input="嗨！",
        temperature=1.9,
        top_p=0.98
    )

    print(response.output_text)

## 2-3 控制回覆格式--生成 JSON

In [ ]:
for _ in range(3):
    response = client.responses.create(
        model="gpt-4.1-nano",
        input="台灣最高的山多高"
    )

    print(response.output_text)

### 生成 JSON 格式

In [ ]:
for _ in range(3):
    response = client.responses.create(
        model="gpt-4.1-nano",
        # instructions="請用 json 格式回覆", # 這無效
        input=[
            # 一定要在 developer 或是 user 訊息中出現 "json"
            {"role": "developer", "content": "使用 json 格式"},
            {"role": "user", "content": "台灣最高的山多高"}
        ],
        text={
            'format': {
                'type': 'json_object'} # 預設為 'text'
            }
    )

    print(response.output_text)

In [ ]:
for _ in range(3):
    response = client.responses.create(
        model="gpt-4.1-nano",
        input=[
            {
                "role":"user",
                "content": "台灣最高的山多高, "
                           "請以如下 json 格式回覆："
                           '{"name":"山的名稱", "height":高度}'
            },
        ],
        text={
            "format": {
                'type': 'json_object'
            }
        }
    )

    print(response.output_text)

### 使用 Pydantic 標準化 JSON 格式生成步驟

In [ ]:
from pydantic import BaseModel, Field, ConfigDict

In [ ]:
class Mountain(BaseModel):
    name: str = Field(description='山的名稱')
    height: int= Field(description='山的高度')

In [ ]:
json_schema = Mountain.model_json_schema()
pprint(json_schema)

In [ ]:
class Mountain(BaseModel):
    # 禁止出現不在 properties 清單中的項目
    model_config = ConfigDict(extra='forbid')
    name: str = Field(description='山的名稱')
    height: int= Field(description='山的高度')

json_schema = Mountain.model_json_schema()
pprint(json_schema)

In [ ]:
for _ in range(3):
    response = client.responses.create(
        model = "gpt-4.1-nano",
        input = "台灣最高的山多高",
        text = {
            "format": {
                "type": "json_schema",
                "name": json_schema["title"],
                "schema": json_schema
            }
        }
    )

    print(response.output_text)

In [ ]:
mountain = Mountain.model_validate_json(
    response.output_text
)
pprint(mountain)
print(mountain.name, mountain.height)

## 2-4 輸入圖片/檔案當提示

### 輸入圖片

火車照片：

|照片一|照片二|
|---|---|
|![](https://flagtech.github.io/images/train1.jpeg)|![](https://flagtech.github.io/images/train2.jpeg)|
|https://flagtech.github.io/images/train1.jpeg|https://flagtech.github.io/images/train1.jpeg|

In [ ]:
photo1_url = "https://flagtech.github.io/images/train1.jpeg"
photo2_url = "https://flagtech.github.io/images/train2.jpeg"

response = client.responses.create(
    model="gpt-4.1-nano",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "圖片裡有什麼？"},
            {
                "type": "input_image",
                "image_url": photo1_url,
                "detail": "high", # low、auto（預設）
            },
        ],
    }],
)

In [ ]:
print(response.output_text)

這張照片是著名的**只見線鐵道橋**（只見川橋梁）冬季景色，位於日本福島縣的**只見線**（JR只見線）。只見線橫跨只見川，被譽為「日本最夢幻鐵路」之一，特別是在大雪覆蓋的冬天，景色非常壯麗。

照片中的橋樑及黃色列車，搭配雪景，非常具有代表性，是攝影愛好者熱門的取景地點。


In [ ]:
pprint(response)

#### 傳送本機的圖片

In [ ]:
!curl "https://flagtech.github.io/images/train2.jpeg" -o train2.jpeg

In [ ]:
import base64

# 把圖檔內容以 base64 編碼的函式
def encode_file(file_path):
    with open(file_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')

In [ ]:
base64_image = encode_file('train2.jpeg')

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": "圖片裡有什麼？"},
                {
                    "type": "input_image",
                    "image_url":
                        f"data:image/jpeg;base64,{base64_image}",
                    'detail': 'high'
                },
            ],
        }
    ],
)

print(response.output_text)

#### 傳送多張圖片

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text":
                    "這兩張圖片裡共通的元素是？"},
                {
                    "type": "input_image",
                    "image_url": photo1_url,
                    "detail": "high",
                },
                {
                    "type": "input_image",
                    "image_url": photo2_url,
                    "detail": "high",
                },
            ],
        }
    ],
)

print(response.output_text)

#### 輸入圖片的計費方式

- [計費說明頁面](https://platform.openai.com/docs/guides/images#calculating-costs)
- [Pricing 頁面](https://openai.com/api/pricing/) FAQ 區最後有輸入圖片的費用計算機
- [驗算用的輔助函式](https://colab.research.google.com/drive/15UqEWFWyRKOSnm7UzgplW4pJ7xYV8t29?usp=sharing)

### 輸入 PDF 檔案

#### 以 Data URL 傳送檔案

In [ ]:
import requests
pdf_url = 'https://coolermaster.egnyte.com/dd/4pPb6Srybx/'
response = requests.get(pdf_url)

In [ ]:
if response.status_code == 200:
    base64_string = base64.b64encode(
        response.content
    ).decode('utf-8')

    attachment = response.headers['content-disposition']
    # attachment;filename="XXX.pdf"
    filename = attachment.split('filename=')[1].strip('"')
else:
    print('下載檔案不成功')

In [ ]:
attachment

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {
            'role': 'user',
            'content': [
                {
                    'type': 'input_file',
                    'filename': filename,
                    'file_data': "data:application/pdf;"
                                 f"base64,{base64_string}"
                },
                {
                    'type': 'input_text',
                    'text': '這個鍵盤的型號以及藍牙配對的方法？'
                }
            ]
        }
    ]
)

In [ ]:
print(response.output_text)
pprint(response.usage)

In [ ]:
pprint(response)

#### 使用網頁介面上傳檔案

PDF 檔：
- https://coolermaster.egnyte.com/dd/4pPb6Srybx/

Dashboard 網址：
- https://platform.openai.com/storage

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {
            'role': 'user',
            'content': [
                {
                    'type': 'input_file',
                    'file_id': "file-CSipLboNkkKJZ2f16YBrA7"
                },
                {
                    'type': 'input_text',
                    'text': '這個鍵盤如何變化燈光效果？'
                }
            ]
        }
    ]
)

print(response.output_text)

根據提供的使用說明，這個鍵盤可以藉由按特定的功能鍵（FN鍵）來切換或調整燈光效果。以下是一些燈光控制的方法摘要：

1. **切換燈光模式**：  
   - 按 FN + DEL/PGUP/PGDN 可以在不同的燈光模式間切換。  
   - 在“藍牙模式”、“2.4 GHz模式”、“有線模式”等不同連線狀態下，都可用此組合切換燈光效果。

2. **調整燈光亮度或效果**：  
   - 一些模式下，按 FN + DEL/PGUP/PGDN 可以調整亮度或啟用特定的燈光效果。  
   - 例如：“請按 FN + DEL/PGUP/PGDN 以啟動燈光效果”或“用 FN + DEL/PGUP/PGDN 切換不同的燈光模式”。

3. **特定燈光效果切換**：  
   - 按 FN + Enter 可切換不同的燈光效果或亮度狀態（如強光或柔和模式）。  
   - 某些描述提到，長按 FN + DEL/PGUP/PGDN 可以進入或退出燈光效果調整狀態。

4. **燈光指示與模式設定**：  
   - 燈光會根據不同的狀態（配對、連線、低電量等）呈現不同的顏色和閃爍方式（例如：紅色閃爍代表低電量）。  

5. **軟體自訂**：  
   - 提到可以使用 Cooler Master 的軟體來自訂低電量模式的燈光百分比，代表可以透過軟體進一步細緻調整燈光效果。

**總結**：  
你可以透過按 FN 與組合鍵（如 DEL/PGUP/PGDN、Enter）來切換燈光模式和亮度，甚至自訂燈光效果。不同的快捷鍵會改變燈光的顏色、閃爍頻率或模式，具體細節可以參考用戶手冊中的燈光控制章節。


In [ ]:
print(response.output_text)

#### 使用 Files API 上傳檔案

In [ ]:
!curl -JO "https://coolermaster.egnyte.com/dd/4pPb6Srybx/"

In [ ]:
!curl "https://coolermaster.egnyte.com/dd/4pPb6Srybx/" \
 -o ck721.pdf

In [ ]:
with open('ck721.pdf', 'rb') as f:
    file_obj = client.files.create(
        file=f,
        purpose='user_data'
    )

In [ ]:
pprint(file_obj)

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {
            'role': 'user',
            'content': [
                {
                    'type': 'input_file',
                    'file_id': file_obj.id
                },
                {
                    'type': 'input_text',
                    'text': '這個鍵盤電量過低時的指示？'
                }
            ]
        }
    ]
)

print(response.output_text)

In [ ]:
client.files.delete(file_obj.id)